In [2]:
import cv2
import numpy as np
from numpy import shape
import matplotlib.pyplot as plt 
from image_extractor import feature_extract, statistic_analysis

image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Agglutinated/image (17).JPG'
# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Brittle/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Purple/image (10).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_PartiallyPurple/image (16).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Brown/image (3).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_White/image (72).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Slaty/image (71).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Slaty/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Moldered/image (7).JPG'
# image_dir = 'D:/Thesis_data/mlp_data/training_img/Compartmentalized_Brown/image(3).JPG'
# image_dir  = '/Users/lochuynhquang/Desktop/Color_Corrected_512x512/Plated_PartiallyPurple/image (10).JPG'

plt.rcParams['figure.dpi'] = 102.46 

In [3]:
extractor = feature_extract()
image_bgr = cv2.imread(image_dir)
# extractor.extract(image_bgr)
extractor.pre_process(image_bgr)
extractor.extract_lbp()


In [4]:
shape(extractor.lbp_hist)

(26,)